##**Q2 Part B**

In [ ]:


from pulp import LpVariable, LpProblem, LpMaximize, lpSum
import pandas as pd

profit_maximization = LpProblem("maximize_GI_profit", LpMaximize)

products = [1, 2]
sale_types = ["cash", "credit"]
sales_price = {(1, "cash"): 40, (1, "credit"): 39.5, (2, "cash"): 52.5, (2, "credit"): 51.5}
production_cost = {1: 35, 2: 45}
inventory_initial = {1: 60, 2: 30}

# Decision Variables
X = {(i, j): LpVariable(f"X_{i}_{j}", lowBound=0) for i in products for j in sale_types}
p = {i: LpVariable(f"p_{i}", lowBound=0) for i in products}
Cash = LpVariable("Cash", lowBound=0)

# Objective Function
revenue = lpSum(sales_price[i, j] * X[i, j] for i in products for j in sale_types)
cost = lpSum(production_cost[i] * p[i] for i in products)
inventory_cost = 0.1 * lpSum(production_cost[i] * (inventory_initial[i] + p[i] - lpSum(X[i, j] for j in sale_types)) for i in products)
profit_maximization += revenue - cost - inventory_cost - 0.05 * Cash

# Constraints
max_sales = {(1, "cash"): 150, (1, "credit"): 100, (2, "cash"): 175, (2, "credit"): 250}
for i in products:
    for j in sale_types:
        profit_maximization += X[i, j] <= max_sales[i, j]

profit_maximization += 2 * p[1] + 4 * p[2] <= 1000

for i in products:
    profit_maximization += inventory_initial[i] + p[i] - lpSum(X[i, j] for j in sale_types) >= 0

profit_maximization += Cash == lpSum(sales_price[i, j] * X[i, j] for i in products for j in sale_types if j == "cash") - lpSum(production_cost[i] * p[i] for i in products)

solution_status = profit_maximization.solve()

solution = {
    "Status": profit_maximization.status,
    "Profit": profit_maximization.objective.value(),
    **{f"X_{i}_{j}": X[i, j].value() for i in products for j in sale_types},
    **{f"p_{i}": p[i].value() for i in products},
    "Cash": Cash.value()
}

columns = list(solution.keys())
values = list(solution.values())
df_solution = pd.DataFrame([values], columns=columns)
df_solution.T

,0
Status,1.000000
Profit,5997.738380
X_1_cash,150.000000
X_1_credit,100.000000
X_2_cash,145.238100
X_2_credit,39.761905
p_1,190.000000
p_2,155.000000
Cash,0.000000


##**Q2 Part D Goal 1**

In [ ]:


from pulp import LpMaximize, LpProblem, LpVariable, lpSum

model = LpProblem(name="maximize_GI_profit_with_goals", sense=LpMinimize)

# Decision variables

# Xij - sales of product i in mode j (cash or credit)
X = {(i, j): LpVariable(f"X{i}{j}", lowBound=0) for i in [1, 2] for j in ['cash', 'credit']}

# Pi - production of product i
P = {i: LpVariable(f"P{i}", lowBound=0) for i in [1, 2]}

# Hi - ending inventory of product i
H = {i: LpVariable(f"H{i}", lowBound=0) for i in [1, 2]}

# Cash on hand at the end of the period
C = LpVariable("C", lowBound=0)

# Deviation variables for goals
D_plus = {i: LpVariable(f"D{i}_plus", lowBound=0) for i in range(1, 4)}
D_minus = {i: LpVariable(f"D{i}_minus", lowBound=0) for i in range(1, 4)}

# Loan taken at the beginning of the six month period
Loan = LpVariable("Loan", lowBound=0)

sales_price = {1: {'cash': 40, 'credit': 39.5}, 2: {'cash': 52.5, 'credit': 51.5}}
production_cost = {1: 35, 2: 45}
inventory_initial = {1: 60, 2: 30}

# Objective Function
model += D_plus[1] + D_minus[1]

# Constraints
max_sales = {1: {'cash': 150, 'credit': 100}, 2: {'cash': 175, 'credit': 250}}
for i in [1, 2]:
    for j in ['cash', 'credit']:
        model += X[i, j] <= max_sales[i][j]


model += 2 * P[1] + 4 * P[2] <= 1000


for i in [1, 2]:
    model += H[i] == inventory_initial[i] + P[i] - lpSum(X[i, j] for j in ['cash', 'credit'])


model += C == lpSum(sales_price[i]['cash'] * X[i, 'cash'] for i in [1, 2]) - lpSum(production_cost[i] * P[i] for i in [1, 2]) - 0.1 * lpSum(production_cost[i] * H[i] for i in [1, 2])


model += lpSum(sales_price[i]['cash'] * X[i, 'cash'] for i in [1, 2]) - lpSum(production_cost[i] * P[i] for i in [1, 2]) - 0.1 * lpSum(production_cost[i] * H[i] for i in [1, 2]) - D_plus[1] + D_minus[1] == 75
model += Loan - 0.75 * lpSum(production_cost[i] * H[i] for i in [1, 2]) <= 0
model += lpSum(sales_price[i][j] * X[i, j] for i in [1, 2] for j in ['cash', 'credit']) - lpSum(production_cost[i] * P[i] for i in [1, 2]) - 0.1 * lpSum(production_cost[i] * H[i] for i in [1, 2]) - 0.05 * C - 1.05 * Loan - D_plus[2] + D_minus[2] == 5997.738357500002
model += lpSum((sales_price[i]['cash'] * X[i, 'cash'] - production_cost[i] * P[i] - 0.1 * production_cost[i] * H[i]) for i in [1, 2]) + lpSum(sales_price[i]['cash'] * X[i, 'credit'] for i in [1, 2]) + lpSum(production_cost[i] * H[i] for i in [1, 2]) - 2 * Loan - D_plus[3] + D_minus[3] == 300

status = model.solve()

result = {
    "Status": model.status,
    **{f"X{i}{j}": X[i, j].value() for i in [1, 2] for j in ['cash', 'credit']},
    **{f"P{i}": P[i].value() for i in [1, 2]},
    **{f"H{i}": H[i].value() for i in [1, 2]},
    "C": C.value(),
    **{f"D{i}_plus": D_plus[i].value() for i in range(1, 4)},
    **{f"D{i}_minus": D_minus[i].value() for i in range(1, 4)},
    "Loan": Loan.value()
}


columns = list(result.keys())
values = list(result.values())
df_result = pd.DataFrame([values], columns=columns)
df_result.T


,0
Status,1.0000
X1cash,1.8750
X1credit,58.1250
X2cash,0.0000
X2credit,30.0000
P1,0.0000
P2,0.0000
H1,0.0000
H2,0.0000
C,75.0000


##**Q2 Part D Goal 2**

In [ ]:


from pulp import LpMaximize, LpProblem, LpVariable, lpSum

model = LpProblem(name="maximize_GI_profit_with_goals", sense=LpMinimize)

# Decision variables

# Xij - sales of product i in mode j (cash or credit)
X = {(i, j): LpVariable(f"X{i}{j}", lowBound=0) for i in [1, 2] for j in ['cash', 'credit']}

# Pi - production of product i
P = {i: LpVariable(f"P{i}", lowBound=0) for i in [1, 2]}

# Hi - ending inventory of product i
H = {i: LpVariable(f"H{i}", lowBound=0) for i in [1, 2]}

# Cash on hand at the end of the period
C = LpVariable("C", lowBound=0)

# Deviation variables for goals
D_plus = {i: LpVariable(f"D{i}_plus", lowBound=0) for i in range(1, 4)}
D_minus = {i: LpVariable(f"D{i}_minus", lowBound=0) for i in range(1, 4)}

# Loan taken at the beginning of the six month period
Loan = LpVariable("Loan", lowBound=0)

sales_price = {1: {'cash': 40, 'credit': 39.5}, 2: {'cash': 52.5, 'credit': 51.5}}
production_cost = {1: 35, 2: 45}
inventory_initial = {1: 60, 2: 30}

# Objective Function
model += D_minus[2]

# Constraints
max_sales = {1: {'cash': 150, 'credit': 100}, 2: {'cash': 175, 'credit': 250}}
for i in [1, 2]:
    for j in ['cash', 'credit']:
        model += X[i, j] <= max_sales[i][j]


model += 2 * P[1] + 4 * P[2] <= 1000


for i in [1, 2]:
    model += H[i] == inventory_initial[i] + P[i] - lpSum(X[i, j] for j in ['cash', 'credit'])


model += C == lpSum(sales_price[i]['cash'] * X[i, 'cash'] for i in [1, 2]) - lpSum(production_cost[i] * P[i] for i in [1, 2]) - 0.1 * lpSum(production_cost[i] * H[i] for i in [1, 2])


model += lpSum(sales_price[i]['cash'] * X[i, 'cash'] for i in [1, 2]) - lpSum(production_cost[i] * P[i] for i in [1, 2]) - 0.1 * lpSum(production_cost[i] * H[i] for i in [1, 2]) - D_plus[1] + D_minus[1] == 75
model += Loan - 0.75 * lpSum(production_cost[i] * H[i] for i in [1, 2]) <= 0
model += D_plus[1] + D_minus[1] == 0
model += lpSum(sales_price[i][j] * X[i, j] for i in [1, 2] for j in ['cash', 'credit']) - lpSum(production_cost[i] * P[i] for i in [1, 2]) - 0.1 * lpSum(production_cost[i] * H[i] for i in [1, 2]) - 0.05 * C - 1.05 * Loan - D_plus[2] + D_minus[2] == 5997.738357500002
model += lpSum((sales_price[i]['cash'] * X[i, 'cash'] - production_cost[i] * P[i] - 0.1 * production_cost[i] * H[i]) for i in [1, 2]) + lpSum(sales_price[i]['cash'] * X[i, 'credit'] for i in [1, 2]) + lpSum(production_cost[i] * H[i] for i in [1, 2]) - 2 * Loan - D_plus[3] + D_minus[3] == 300

status = model.solve()

result = {
    "Status": model.status,
    **{f"X{i}{j}": X[i, j].value() for i in [1, 2] for j in ['cash', 'credit']},
    **{f"P{i}": P[i].value() for i in [1, 2]},
    **{f"H{i}": H[i].value() for i in [1, 2]},
    "C": C.value(),
    **{f"D{i}_plus": D_plus[i].value() for i in range(1, 4)},
    **{f"D{i}_minus": D_minus[i].value() for i in range(1, 4)},
    "Loan": Loan.value()
}


columns = list(result.keys())
values = list(result.values())
df_result = pd.DataFrame([values], columns=columns)
df_result.T

,0
Status,1.000000
X1cash,150.000000
X1credit,100.000000
X2cash,146.666670
X2credit,38.333333
P1,190.000000
P2,155.000000
H1,0.000000
H2,0.000000
C,75.000000


##**Q3 Part A**

In [ ]:


from pulp import *
import pandas as pd

prob = LpProblem("Maximize_Consecutive_Days_Off", LpMaximize)

# Decision variables
# x[i][j] is 1 if employee i works on day j
x = LpVariable.dicts("x", (range(1, 12), range(1, 8)), cat='Binary')

# Auxiliary variables
# y[i][j] is 1 if employee i has a day off on j and j+1, including Sunday to Monday
y = LpVariable.dicts("y", (range(1, 12), range(1, 8)), cat='Binary')

# Objective Function
prob += lpSum(y[i][j] for i in range(1, 12) for j in range(1, 8)), "Objective"

# Constraints

# Each employee only has 4 working days
for i in range(1, 12):
    prob += lpSum(x[i][j] for j in range(1, 8)) == 4

# Swan Requires atleast 7 line employees from Monday to Friday
for j in range(1, 6):
    prob += lpSum(x[i][j] for i in range(1, 12)) >= 7

# Swan Requires atleast 3 line employees on Saturday and Sunday
for j in range(6, 8):
    prob += lpSum(x[i][j] for i in range(1, 12)) >= 3

# y[i][j] calculation based on x[i][j]
for i in range(1, 12):
    for j in range(1, 7):
        prob += y[i][j] <= 1 - x[i][j]
        prob += y[i][j] <= 1 - x[i][j + 1]
        prob += y[i][j] >= 1 - (x[i][j] + x[i][j + 1])
    prob += y[i][7] <= 1 - x[i][7]
    prob += y[i][7] <= 1 - x[i][1]
    prob += y[i][7] >= 1 - (x[i][7] + x[i][1])

prob.solve()

# Working Schedule
schedule_results_x = {}
for i in range(1, 12):
    schedule_results_x[f"Employee_{i}"] = {}
    for j in range(1, 8):
        schedule_results_x[f"Employee_{i}"][f"Day_{j}"] = x[i][j].varValue

# Consecutive Off Days Schedule
schedule_results_y = {}
for i in range(1, 12):
    schedule_results_y[f"Employee_{i}"] = {}
    for j in range(1, 8):  # Adjusted to include Sunday to Monday
        schedule_results_y[f"Employee_{i}"][f"Day_{j}"] = y[i][j].varValue

# Total Consecutive Off Days Adjusting for 1 Extra in Calculation as y[i][j] == 2 for 3 Consecutive Offs
schedule_results_c = {}
for i in range(1, 12):
    consecutive_days_off = sum(y[i][j].varValue for j in range(1, 8))  # Adjusted to include Sunday to Monday
    consecutive_days_off += 1  # Add 1 to calculate the actual number of consecutive days off from a binary variable
    schedule_results_c[f"Employee_{i}"] = consecutive_days_off

adjusted_objective_value = sum(schedule_results_c[employee] for employee in schedule_results_c)


df_schedule = pd.DataFrame.from_dict(schedule_results_x, orient='index')
df_schedule.columns = ['Day_{}'.format(j) for j in range(1, 8)]
df_consecutive_off = pd.DataFrame.from_dict(schedule_results_y, orient='index')
df_consecutive_off.columns = ['Day_{}'.format(j) for j in range(1, 8)]  # Adjusted to include Sunday to Monday
df_total_consecutive_off = pd.DataFrame.from_dict(schedule_results_c, orient='index', columns=['Total Consecutive Days Off'])
df_total_consecutive_off.loc['Sum Total'] = df_total_consecutive_off.sum()


print("Work Schedule (x)")
print(df_schedule)
print("\nConsecutive Days Off (y)")
print(df_consecutive_off)
print("\nTotal Consecutive Days Off (c)")
print(df_total_consecutive_off)
print("\nStatus:", LpStatus[prob.status])
print("Original Value of Objective Function:", pulp.value(prob.objective))
print("Adjusted Value of Objective Function:", adjusted_objective_value)

Work Schedule (x)
             Day_1  Day_2  Day_3  Day_4  Day_5  Day_6  Day_7
Employee_1     1.0    1.0    1.0    1.0    0.0    0.0    0.0
Employee_2     0.0    1.0    1.0    1.0    1.0    0.0    0.0
Employee_3     0.0    1.0    1.0    1.0    1.0    0.0    0.0
Employee_4     1.0    0.0    0.0    0.0    1.0    1.0    1.0
Employee_5     1.0    0.0    0.0    0.0    1.0    1.0    1.0
Employee_6     0.0    1.0    1.0    1.0    1.0    0.0    0.0
Employee_7     1.0    1.0    1.0    1.0    0.0    0.0    0.0
Employee_8     1.0    1.0    1.0    0.0    0.0    0.0    1.0
Employee_9     1.0    0.0    0.0    0.0    1.0    1.0    1.0
Employee_10    0.0    1.0    1.0    1.0    1.0    0.0    0.0
Employee_11    1.0    1.0    1.0    1.0    0.0    0.0    0.0

Consecutive Days Off (y)
             Day_1  Day_2  Day_3  Day_4  Day_5  Day_6  Day_7
Employee_1     0.0    0.0    0.0    0.0    1.0    1.0    0.0
Employee_2     0.0    0.0    0.0    0.0    0.0    1.0    1.0
Employee_3     0.0    0.0    0.0    0.0  